## TensorFlow深层神经网络

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》

### 4.1 深度学习与深层神经网络

维基百科对深度学习的精确定义：“一类通过多层非线性变换对高复杂性数据建模算法的合集”。  
深度学习两个非常重要的特性——多层、非线性  
只通过线性变换，任意层的全连接神经网络和单层神经网络模型的表达能力没有任何区别，且都是线性模型。可通过TensorFlow Playground实验验证。  

TensorFlow提供了7种不同的非线性激活函数，比较常用的有：

    tf.nn.relu
    tf.sigmoid
    tf.tanh

使用了激活函数和偏置项的神经网络前向传播算法：

    a = tf.nn.relu(tf.matmul(x,w1) + biases1)
    y = tf.nn.relu(tf.matmul(a,w2) + biases2)

深层神经网络实际上有组合特征提取的功能，这个特性对于解决不易提取特征向量的问题（比如图片识别、语音识别等）有很大的帮助。

### 4.2 损失函数定义

通过神经网络解决多分类问题最常用的方法是设置n个输出节点，n为类别的个数。  
交叉熵（cross entropy）是判断一个输出向量和期望的向量的接近程度的常用方法之一，刻画的是两个概率分布之间的距离，然而神经网络的输出却不一定是一个概率分布。Softmax回归可以将神经网络前向传播得到的结果变成概率分布。Softmax回归本身可以作为一个学习算法来优化分类结果，但在TensorFlow中softmax回归的参数被去掉了，它只是一层额外的处理层，将神经网络的输出变成一个概率分布。   
从交叉熵公式可以看出H(p,q)≠H(q,p)。p代表正确答案，q代表预测值，H(p,q)刻画通过概率分布q来表达概率分布p的困难程度。

In [ ]:
# 实现交叉熵
cross_entropy = -tf.reduce_mean(
    y_ * tf.log(tf.clip_by_value(y,1e-10,1.0)))
#y_代表正确结果，y代表预测结果

In [1]:
# tf.clip_by_value函数可以将一个张量中的数值限制在一个范围之内，以避免运算错误（如log0）
import tensorflow as tf
v = tf.constant([[1.0,2.0,3.0],[4.0,5.0,6.0]])
with tf.Session() as sess:
    print(tf.clip_by_value(v,2.5,4.5).eval())
    #v中小于2.5用2.5来替代，大于4.5用4.5来替代

[[2.5 2.5 3. ]
 [4.  4.5 4.5]]


In [2]:
# tf.log函数对张量中所有元素依次求对数
v = tf.constant([1.0,2.0,3.0])
with tf.Session() as sess:
    print(tf.log(v).eval())

[0.        0.6931472 1.0986123]


In [3]:
# *不是矩阵乘法，而是对应元素相乘。矩阵乘法用tf.matmul
v1 = tf.constant([[1,2],[3,4]])
v2 = tf.constant([[5,6],[7,8]])
with tf.Session() as sess:
    print((v1*v2).eval())
    print(tf.matmul(v1,v2).eval())

[[ 5 12]
 [21 32]]
[[19 22]
 [43 50]]


根据交叉熵的公式，应该将每行中的m个结果相加得到所有样例的交叉熵，再对这n行平均得到一个batch的平均交叉熵。但因为分类问题的类别数量是不变的，所以可以直接对整个矩阵做平均而不改变计算结果的意义。这样的方式可以使整个程序更加简洁。 

In [5]:
# tf.reduce_mean函数
v=tf.constant([[1.0,2.0,3.0],[4.0,5.0,6.0]])
with tf.Session() as sess:
    print(tf.reduce_mean(v).eval())

3.5


交叉熵一般会和softmax一起使用，所以tf对这两个功能进行了统一封装，提供了下面函数：

    tf.nn.softmax_cross_entropy_with_logits() 

在只有一个正确答案的分类问题中，tf提供了下面函数进一步加速计算过程：

    tf.nn.sparse_softmax_cross_entropy_with_logits()

In [ ]:
# 实现使用softmax回归之后的交叉熵损失函数
cross_entropy=tf.nn.softmax_cross_entropy_with_logits(y,y_)

对于回归问题，最常用的损失函数是均方误差(MSE)。 

In [ ]:
# 均方误差
mse = tf.reduce_mean(tf.square(y_-y))

TF不仅支持经典的损失函数，还可以优化任意的自定义损失函数。以预测商品销量问题为例：

In [ ]:
# 自定义损失函数
loss = tf.reduce_sum(tf.where(tf.greater(v1,v2),
                              (v1-v2)*a,(v2-v1)*b))

tf.greater(A,B)的输入是两个张量，比较这两个输入张量中每一个元素的大小，并返回比较结果。  
tf.where(a,b,c)函数中有三个参数，第一个为选择条件依据，True时tf.where函数会选择第二个参数中的值，False使用第三个参数中的值。注意：判断和选择都是在元素级别进行。

In [6]:
# tf.where函数和tf.greater函数的用法
v1=tf.constant([1,2,3,4])
v2=tf.constant([4,3,2,1])
with tf.Session() as sess:
    print(tf.greater(v1,v2).eval())
    print(tf.where(tf.greater(v1,v2),v1,v2).eval())

[False False  True  True]
[4 3 3 4]


在下列程序中，实现了一个拥有两个输入节点，一个输出节点，没有隐藏层的神经网络：

In [11]:
import tensorflow as tf
from numpy.random import RandomState

batch_size=8

#两个输入节点
x=tf.placeholder(tf.float32,shape=(None,2),name="x-input")
#回归问题一般只有一个输出节点
y_=tf.placeholder(tf.float32,shape=(None,1),name="y-input")

#定义一个单层的神经网络前向传播的过程，这里是简单加权和
w1=tf.Variable(tf.random_normal([2,1],stddev=1,seed=1))
y=tf.matmul(x,w1)

#定义预测多了和预测少了的成本
loss_less=10
loss_more=1
loss=tf.reduce_sum(tf.where(tf.greater(y,y_),
                            (y-y_)*loss_more,
                            (y_-y)*loss_less))
#优化器
train_step=tf.train.AdamOptimizer(0.001).minimize(loss)

#通过随机数生成一个模拟数据集
rdm=RandomState(1)
dataset_size=128
X=rdm.rand(dataset_size,2)

#设置回归的正确值为两个输入的和加上一个随机量。加上随机量是为了
#加入不可预测的噪音，否则不同损失函数的意义就不大了，
#因为不同损失函数都会在能完全预测正确的时候最低。
#一般来说，噪音为一个均值为0的小量，所以这里噪声设置为-0.05~0.05的随机数
Y=[[x1+x2+rdm.rand()/10.0-0.05] for (x1,x2) in X]

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    STEPS=5000
    for i in range(STEPS):
        start=(i*batch_size)%dataset_size
        end=min(start+batch_size,dataset_size)
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
    print(sess.run(w1))

[[1.019347 ]
 [1.0428089]]


也就是说预测函数是1.02x1+1.04x2，这要比x1+x2大，因为在损失函数中指定预测少了的损失更大。  
通过这个样例可以感受到，对于相同的神经网络，不同的损失函数会对训练得到的模型产生重要影响。

### 4.3 神经网络优化算法

梯度下降法：优化单个参数值  
反向传播算法：在所有参数上使用梯度下降法 
参数更新公式：
$$
\theta_{n+1}=\theta_n-\eta\frac{\partial}{\partial \theta_n}J(\theta_n)
$$
其中$\eta$为学习率，$J(\theta)$为损失函数  

神经网络的优化过程可以分为两个阶段：  
第一阶段：通过前向传播算法得到预测值，并将预测值和真实值做对比得到两者之间的差距  
第二阶段：通过反向传播算法计算损失函数对每一个参数的梯度，再根据梯度和学习率使用梯度下降法更新每一个参数  
注意：梯度下降法不能确保被优化的函数达到全局最优解。参数的初始值很大程度影响最后的结果。只有当损失函数为凸函数时，梯度下降法才能保证达到全局最优解。

梯度下降算法与随机梯度下降算法的折中——每次计算一小部分训练数据的损失函数。这一小部分数据称之为batch。通过矩阵运算，每次在一个batch上优化神经网络的参数并不会比单个数据慢太多，另一方面，使用一个batch可以大大减少收敛所需要的迭代次数，同时可以使收敛的结果更加接近梯度下降的效果。 

下面给出神经网络大致遵循的训练过程：

In [ ]:
#神经网络的一般训练过程
batch_size=n

#每次读取一小部分数据作为当前的训练数据来执行反向传播算法
x=tf.placeholder(tf.float32,shape=(batch_size,2),name="x-input")
y_=tf.placeholder(tf.float32,shape=(batch_size,1),name="y-input")

#定义神经网络结构和优化算法
loss=...
train_step=tf.train.AdamOptimizer(0.001).minimize(loss)

#训练神经网络
with tf.Session() as sess:
     #参数初始化
    tf.global_variables_initializer().run()
    #迭代更新参数
    for i in range(STEPS):
        #准备batch_size个训练数据，一般将所有训练数据随机打乱后再选取可以得到
        #更好的优化效果
        start,end=...
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})